In [117]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error, r2_score

In [118]:
df = pd.read_csv("RPGF3Results.csv")
df = df[df['Category: Developer Ecosystem'] == 1]
df.shape

(305, 58)

In [119]:
df.drop(columns=['Project_ID', 'Result: Quorum Reached', 'Meta: Applicant Type','Meta: Website','Meta: Bio','Meta: Payout Address',	'Category: Collective Governance','Category: Developer Ecosystem','Category: End User Experience and Adoption','Category: OP Stack','Keywords: Base','Keywords: Farcaster','Keywords: Zora','Link: Contract on Base','Link: Contract on OP Mainnet','Link: Dune','Link: Flipside','Link: GitHub','Link: GitHub (duneanalytics)','Link: GitHub (ethereum)','Link: GitHub (ethereum-optimism)','Link: NPM Package','Link: Optimism Gov','Link: Substack','Link: Twitter','GTP: VC Funding Amount','GTP: Has Token', 'GTP: Has VC Funding','Meta: Project Image'], inplace=True)
df.to_csv("Cleaned_Retro.csv")

Machine Learning Support Vector Regression (SVR)

In [120]:
df_SVR = df[['Result: # Ballots', 'Result: Received OP','OSO: # GitHub Repos','OSO: Total Stars','OSO: Total Forks','OSO: Total Contributors','OSO: Contributors Last 6 Months','OSO: Avg Monthly Active Devs Last 6 Months', 'OSO: Total Onchain Users', 'OSO: Onchain Users Last 6 Months', 'OSO: Total Downloads', 'OSO: Downloads Last 6 Months']]	
df_SVR.fillna(0, inplace=True)
# df_SVR.reset_index(inplace=True)
df_SVR.to_csv("RetroML.csv")

/var/folders/rg/45gtrwls705094nr5_p2f7980000gn/T/ipykernel_17478/1227122324.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_SVR.fillna(0, inplace=True)


In [109]:
sample_data = {
    'Result: # Ballots': [92],
    'OSO: # GitHub Repos': [270.0],
    'OSO: Total Stars': [435090.0],
    'OSO: Total Forks': [18269.0],
    'OSO: Total Contributors': [12072.0],
    'OSO: Contributors Last 6 Months': [1256.0],
    'OSO: Avg Monthly Active Devs Last 6 Months': [49.833333333333336],
    'OSO: Total Onchain Users': [0.0],
    'OSO: Onchain Users Last 6 Months': [0.0],
    'OSO: Total Downloads': [0.0],
    'OSO: Downloads Last 6 Months': [0.0]
}
new_data = pd.DataFrame(sample_data)

## Bad Model SVR

In [110]:
X = df_SVR.drop(columns=['Result: Received OP'])
y = df_SVR['Result: Received OP']

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

svr_model = SVR(kernel='sigmoid')

svr_model.fit(X_scaled, y)

new_data_scaled = scaler.transform(new_data)

new_predictions = svr_model.predict(new_data_scaled)

print("Predictions for 'Received OP' on new data:")
print(new_predictions)

Predictions for 'Received OP' on new data:
[44814.22162497]


In [111]:
from sklearn.ensemble import RandomForestRegressor

## Good Model Random Forest

In [112]:
X = df_SVR.drop(columns=['Result: Received OP'])
y = df_SVR['Result: Received OP']

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

rf_model = RandomForestRegressor(n_estimators=200, random_state=42)

rf_model.fit(X_scaled, y)

new_data_scaled = scaler.transform(new_data)

new_predictions = rf_model.predict(new_data_scaled)

print("Predictions for 'Received OP' on new data using Random Forest Regression:")
print(new_predictions)

Predictions for 'Received OP' on new data using Random Forest Regression:
[542616.59945]


## lightgbm Model

In [113]:
import lightgbm as lgb

In [121]:
X = df_SVR.drop(columns=['Result: Received OP'])
y = df_SVR['Result: Received OP']

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

lgb_model = lgb.LGBMRegressor()

lgb_model.fit(X_scaled, y)

new_data_scaled = scaler.transform(new_data)

new_predictions = lgb_model.predict(new_data_scaled)

print("Predictions for 'Received OP' on new data using LightGBM Regression:")
print(new_predictions)

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000699 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 335
[LightGBM] [Info] Number of data points in the train set: 305, number of used features: 11
[LightGBM] [Info] Start training from score 61288.599747
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain

## XGBoost Model

In [122]:
import xgboost as xgb

In [123]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

xgb_model = xgb.XGBRegressor()

# Fit the model to the entire dataset
xgb_model.fit(X_scaled, y)

new_data_scaled = scaler.transform(new_data)
new_predictions = xgb_model.predict(new_data_scaled)

print("Predictions for 'Received OP' on new data using XGBoost Regression:")
print(new_predictions)

Predictions for 'Received OP' on new data using XGBoost Regression:
[399467.84]
